In [3]:
import sys
sys.path.append(r"C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool") 
from MatchSys import config


from MatchSys import MatchSys
ms = MatchSys(
        name='teat_sys',
        ltp_model_path=r'C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool\data\LtpModel',
        database_uri='sqlite:///data/db.sqlite3',
        text_vec_model_path=r'C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool\data\model.pkl',
        vector_similarity_rate=0.7,
        most_similar_number=10,
        vector_match_times=5,

    )

Starting initialization of MatchSys
Loading weights from local directory


In [5]:
ms.docvector_tool.model.infer_vector('我 喜欢 你'.split(' '))

array([ 0.08856113,  0.07700427, -0.23418705, -0.32499823, -0.00423443,
        0.08868691, -0.02464413, -0.28095624, -0.09659621,  0.06616449,
        0.09258802,  0.08390011,  0.04215227, -0.10699629,  0.2735826 ,
        0.03667052, -0.01707674,  0.21364781,  0.20680977,  0.18371937,
        0.3358171 ,  0.10981559,  0.21698456, -0.11844297, -0.09269717,
        0.24849229,  0.35065418, -0.40114224, -0.18850662,  0.02793114,
       -0.15105368,  0.02940254,  0.4180297 ,  0.05620399, -0.1961256 ,
       -0.06960765,  0.16879193,  0.48983908, -0.255641  ,  0.05271592,
        0.19652812,  0.08155172, -0.08942468,  0.26909223,  0.08594631,
        0.253106  ,  0.2822567 , -0.17792344,  0.05222271,  0.1041132 ,
       -0.03761753, -0.1633152 ,  0.13537948, -0.16662498,  0.05840633,
       -0.1853878 , -0.12461486,  0.18324322, -0.19628051,  0.08837903,
       -0.01257097, -0.12914127,  0.06335146,  0.01610004, -0.06107762,
        0.15930422,  0.05674284, -0.13796331,  0.08233408,  0.32

In [7]:
sims = ms.docvector_tool.model.dv.most_similar([ms.docvector_tool.model.infer_vector('我 喜欢 你'.split(' '))],topn=2)
for sim in sims:
    print(sim)

('1724663540608012290', 0.7994190454483032)
('1724663624338903040', 0.7379180788993835)
